# Calculate SF on 100km slice of QG outputs

In [1]:
# imports
from importlib import reload
import os

import xarray

import numpy as np
# import fsspec
import matplotlib
import matplotlib.pyplot as plt
import gsw_xarray as gsw
from xhistogram.xarray import histogram

from dask.diagnostics import ProgressBar

from strucFunct2_ai import timescale

from profiler import gliderdata
from profiler import profilerpairs
from cugn import io as cugn_io
from cugn import utils as cugn_utils
from cugn import plotting as cugn_plotting

import qg_utils
import qg_100km
import strucFunct2_ai

# Load up

In [2]:
qg, mSF_15 = qg_utils.load_qg()

# Cut down grid

In [3]:
xlim = (300,400)
ylim = (300,400)

# Load
qg, Udsn = qg_utils.load_last_time()

iregion_x = np.where((qg.x >= xlim[0]*1e3) & (qg.x < xlim[1]*1e3))[0]
iregion_y = np.where((qg.y >= ylim[0]*1e3) & (qg.y < ylim[1]*1e3))[0]

# Cut down Usdn
ndays = 60
Udsn2 = Udsn.isel(x=iregion_x, y=iregion_y, time=np.arange(1))
Udsn = Udsn.isel(x=iregion_x, y=iregion_y, time=0)


In [4]:
SF_33 = xarray.load_dataset('Output/SF_region_x300_y300_60days.nc')
SF_33 = SF_33.isel(time=0)
assert SF_33.time == Udsn.time
assert SF_33.time == Udsn2.time.values[0]

In [5]:
Udsn2

<xarray.Dataset> Size: 19kB
Dimensions:  (time: 1, y: 25, x: 25, lev_mid: 1, l: 256, k: 129)
Coordinates:
  * time     (time) float64 8B 6.064e+08
    lev      int64 8B 1
  * lev_mid  (lev_mid) float64 8B 1.5
  * x        (x) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * y        (y) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * l        (l) float64 2kB 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
  * k        (k) float64 1kB 0.0 6.283e-06 1.257e-05 ... 0.000798 0.0008042
Data variables:
    q        (time, y, x) float64 5kB dask.array<chunksize=(1, 25, 25), meta=np.ndarray>
    u        (time, y, x) float64 5kB dask.array<chunksize=(1, 25, 25), meta=np.ndarray>
    v        (time, y, x) float64 5kB dask.array<chunksize=(1, 25, 25), meta=np.ndarray>
Attributes: (12/23)
    pyqg:beta:       1.5e-11
    pyqg:delta:      0.25
    pyqg:del2:       0.8
    pyqg:dt:         3600.0
    pyqg:filterfac:  23.6
    pyqg:L:          1000000.0
    ...              ...
    pyqg:tc:         0
    pyqg:tmax:       622080000.0
    pyqg:twrite:     10000
    pyqg:W:          1000000.0
    title:           pyqg: Python Quasigeostrophic Model
    reference:       https://pyqg.readthedocs.io/en/latest/index.html

# Calculate with Miguel

In [34]:
maxcorr = 30
shiftdim = 'x','y'
grid = 'm'
reload(strucFunct2_ai)
SFtest = strucFunct2_ai.calculateSF_2(Udsn2, maxcorr, shiftdim, grid)

[########################################] | 100% Completed | 17.03 ss


Concatenating results:   0%|          | 0/899 [00:00<?, ?it/s]

In [35]:
SFtest

<xarray.Dataset> Size: 13MB
Dimensions:  (x: 25, y: 25, time: 1, dcorr: 899)
Coordinates:
    lev      int64 8B 1
  * x        (x) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * y        (y) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * time     (time) float64 8B 6.064e+08
Dimensions without coordinates: dcorr
Data variables:
    dr       (dcorr, x, y, time) float64 4MB nan 3.906e+03 3.906e+03 ... nan nan
    ulls     (dcorr, x, y, time) float64 4MB nan -0.002158 ... nan nan
    utts     (dcorr, x, y, time) float64 4MB nan -0.01194 -0.01653 ... nan nan

In [36]:
np.sum(np.isfinite(SFtest.dr.values))

np.int64(105000)

# Calculate

## Loop on x,y

In [43]:
xv = Udsn.x.values
yv = Udsn.y.values
u = Udsn.u.values
v = Udsn.v.values

In [58]:
# WARNING -- This double counts!!
#
drs = []
dus = []
ixx = np.outer(np.ones(yv.size),np.arange(xv.size))
iyy = np.outer(np.arange(yv.size),np.ones(xv.size))
for ix in range(Udsn.x.size):
    for iy in range(Udsn.y.size):
        dx = np.outer(np.ones(yv.size), xv[ix] - xv)
        dy = np.outer(yv[iy] - yv, np.ones(xv.size))
        #
        dU = u[iy,ix] - u
        dV = v[iy,ix] - v
        #dU = u[ix,iy] - u
        #dV = v[ix,iy] - v
        #
        r2 = dx**2 + dy**2
        dr = np.sqrt(r2)
        # duL
        norm = dr
        rx = dx/norm
        ry = dy/norm
        ull = rx*dU + ry*dV
        # Drop the nan 
        keep = np.isfinite(ull)
        # Don't double count
        #keep &= ixx >= ix
        #keep &= iyy >= iy
        # Keep only good ones
        dr = dr[keep].flatten()
        ull = ull[keep].flatten()
        # Save
        drs += dr.tolist()
        dus += ull.tolist()
#
drs = np.array(drs)
dus = np.array(dus)

/tmp/ipykernel_897397/3347401944.py:21: RuntimeWarning: invalid value encountered in divide
  rx = dx/norm
/tmp/ipykernel_897397/3347401944.py:22: RuntimeWarning: invalid value encountered in divide
  ry = dy/norm


In [59]:
drs, dus

(array([ 3906.25,  7812.5 , 11718.75, ..., 11718.75,  7812.5 ,  3906.25]),
 array([ 0.00075308,  0.00152377,  0.00477524, ..., -0.0048605 ,
        -0.0078944 , -0.00445415]))

In [60]:
imax = drs >= drs.max()
np.sum(imax)

np.int64(4)

In [61]:
drs[imax], dus[imax]

(array([132582.52147248, 132582.52147248, 132582.52147248, 132582.52147248]),
 array([-0.09811021,  0.02767934,  0.02767934, -0.09811021]))

In [7]:
ok = np.isfinite(SFtest.dr.data.flatten())
SF_dr = SFtest.dr.data.flatten()[ok] 
SF_ulls = SFtest.ulls.data.flatten()[ok]

In [8]:
imax2 = SF_dr >= drs.max()
SF_dr[imax2], SF_ulls[imax2]

NameError: name 'drs' is not defined

In [37]:
imax3 = np.nanargmax(SFtest.dr.data)
imax3

np.int64(464999)

In [38]:
25*25*899

561875

In [39]:
np.unravel_index(imax3, (899,25,25))

(np.int64(743), np.int64(24), np.int64(24))

In [40]:
SFtest.dr.data.flatten()[imax3]
SFtest.ulls.data.flatten()[imax3]

np.float64(-0.09811020902559855)

In [16]:
SFtest.dr.data[743,24,24], SFtest.ulls.data[743,24,24]

(array([132582.52147248]), array([0.01429679]))

# Let's just do the corners

In [28]:
drmax = np.sqrt((xv[-1]-xv[0])**2 + (yv[-1]-yv[0])**2)
drmax

np.float64(132582.52147247767)

## Check the corners

In [44]:
# Lower left, upper right
ix0, ix1 = 0, -1
iy0, iy1 = 0, -1
# Upper left, lower right
#ix0, ix1 = 0, -1
#iy0, iy1 = -1, 0

dx = xv[ix1]-xv[ix0]
dy = yv[iy1]-yv[iy0]
#
dr = np.sqrt(dx**2 + dy**2)
dx = dx/dr
dy = dy/dr
dx, dy, dr

(np.float64(0.7071067811865475),
 np.float64(0.7071067811865475),
 np.float64(132582.52147247767))

In [45]:
du = u[iy1,ix1] - u[iy0,ix0]
dv = v[iy1,ix1] - v[iy0,ix0]
du, dv

(np.float64(-0.07948375451568254), np.float64(-0.0592650336955782))

In [46]:
uL = dx*du + dy*dv
uL

np.float64(-0.09811020902559855)

In [40]:
SFtest

<xarray.Dataset> Size: 13MB
Dimensions:  (x: 25, y: 25, time: 1, dcorr: 899)
Coordinates:
    lev      int64 8B 1
  * x        (x) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * y        (y) float64 200B 3.027e+05 3.066e+05 ... 3.926e+05 3.965e+05
  * time     (time) float64 8B 6.064e+08
Dimensions without coordinates: dcorr
Data variables:
    dr       (dcorr, x, y, time) float64 4MB nan 3.906e+03 3.906e+03 ... nan nan
    ulls     (dcorr, x, y, time) float64 4MB nan -0.002158 ... nan nan
    utts     (dcorr, x, y, time) float64 4MB nan -0.01194 -0.01653 ... nan nan

In [38]:
du = Udsn.u[-1,-1].values- Udsn.u[0,0].values 
dv = Udsn.v[-1,-1].values - Udsn.v[0,0].values 
du, dv

(np.float64(-0.07948375451568254), np.float64(-0.0592650336955782))

In [39]:
du/np.sqrt(2) + dv/np.sqrt(2)

np.float64(-0.09811020902559857)

# Bin up

In [78]:
# Defines distance bins 
dr = 5000 # meters
rbins = np.arange(0, 1.3e5, dr) # 130 km
mid_rbins = 0.5*(rbins[:-1] + rbins[1:])

In [79]:
rbins.size

26

In [80]:
ibin = 20
in_rbin = (drs > rbins[ibin]) & (drs <= rbins[ibin+1])
#
avg_uL = np.mean(dus[in_rbin])
avg_uL

np.float64(-0.02751662766179287)

## Check

In [81]:
rbins[ibin], mid_rbins[ibin]

(np.float64(100000.0), np.float64(102500.0))

In [82]:
SF_33.ulls[ibin]

<xarray.DataArray 'ulls' ()> Size: 8B
array(0.07266115)
Coordinates:
    lev        int64 8B 1
    time       float64 8B 6.064e+08
    mid_rbins  float64 8B 1.025e+05
Attributes:
    long_name:  $\delta u_L$ averaged for all orientations
    units:      m/s

----

In [74]:
SF_33

<xarray.Dataset> Size: 1kB
Dimensions:    (mid_rbins: 25)
Coordinates:
    lev        int64 8B 1
    time       float64 8B 6.064e+08
  * mid_rbins  (mid_rbins) float64 200B 2.5e+03 7.5e+03 ... 1.175e+05 1.225e+05
Data variables:
    dr         (mid_rbins) float64 200B 2.5e+03 7.5e+03 ... 1.175e+05 1.225e+05
    ulls       (mid_rbins) float64 200B 0.001006 0.005836 ... 0.02854 0.01232
    utts       (mid_rbins) float64 200B 0.002351 0.004937 ... 0.1065 0.1021
    du2        (mid_rbins) float64 200B 6.48e-05 0.0005592 ... 0.0006267
    du3        (mid_rbins) float64 200B 1.558e-07 1.467e-05 ... 2.809e-05
Attributes:
    description:  Structure Function Dataset
    Model:        Two-layer QG Turbulence

----

# Step by step

In [17]:
743 // 30

24

In [31]:
dcorr1 = 24
dcorr2 = 24
shiftdim = 'x','y'
grid = 'm'
Udata = Udsn2.u
Vdata = Udsn2.v

Xg, Yg, tg = xarray.broadcast(Udsn2.x, Udsn2.y, Udsn2.time)

#Udata = Udsn2.u.chunk({'x': 256, 'y': 256, 'time': 1})
#Vdata = Udsn2.v.chunk({'x': 256, 'y': 256, 'time': 1})

reload(strucFunct2_ai)
dr, ull, utt  = strucFunct2_ai.process_dcorr(dcorr1, dcorr2, Udata, Vdata, Xg, Yg, grid)

In [32]:
dr.data.flatten()[-1], ull.data.flatten()[-1]

(np.float64(132582.52147247767),
 dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>)

In [33]:
ull.data

dask.array<add, shape=(25, 25, 1), dtype=float64, chunksize=(25, 25, 1), chunktype=numpy.ndarray>